# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff20c70bee0>
├── 'a' --> tensor([[-0.8564,  0.8907, -0.9702],
│                   [ 1.3878,  0.9042, -0.9959]])
└── 'x' --> <FastTreeValue 0x7ff20c70b070>
    └── 'c' --> tensor([[ 1.2597, -0.6361, -0.7958, -1.6428],
                        [-0.8940, -0.9450,  1.4342, -0.4739],
                        [-1.9996, -1.7055, -0.7922, -0.5828]])

In [4]:
t.a

tensor([[-0.8564,  0.8907, -0.9702],
        [ 1.3878,  0.9042, -0.9959]])

In [5]:
%timeit t.a

77.5 ns ± 0.161 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff20c70bee0>
├── 'a' --> tensor([[-0.8796, -0.5286, -1.1626],
│                   [ 1.2346,  0.3173,  0.8797]])
└── 'x' --> <FastTreeValue 0x7ff20c70b070>
    └── 'c' --> tensor([[ 1.2597, -0.6361, -0.7958, -1.6428],
                        [-0.8940, -0.9450,  1.4342, -0.4739],
                        [-1.9996, -1.7055, -0.7922, -0.5828]])

In [7]:
%timeit t.a = new_value

83 ns ± 0.206 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8564,  0.8907, -0.9702],
               [ 1.3878,  0.9042, -0.9959]]),
    x: Batch(
           c: tensor([[ 1.2597, -0.6361, -0.7958, -1.6428],
                      [-0.8940, -0.9450,  1.4342, -0.4739],
                      [-1.9996, -1.7055, -0.7922, -0.5828]]),
       ),
)

In [10]:
b.a

tensor([[-0.8564,  0.8907, -0.9702],
        [ 1.3878,  0.9042, -0.9959]])

In [11]:
%timeit b.a

69.7 ns ± 0.307 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1395,  0.1545,  1.1544],
               [ 1.4234,  0.5979,  0.3851]]),
    x: Batch(
           c: tensor([[ 1.2597, -0.6361, -0.7958, -1.6428],
                      [-0.8940, -0.9450,  1.4342, -0.4739],
                      [-1.9996, -1.7055, -0.7922, -0.5828]]),
       ),
)

In [13]:
%timeit b.a = new_value

587 ns ± 1.32 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

969 ns ± 1.73 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.1 µs ± 33 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

168 µs ± 549 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

166 µs ± 334 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff162396e50>
├── 'a' --> tensor([[[-0.8564,  0.8907, -0.9702],
│                    [ 1.3878,  0.9042, -0.9959]],
│           
│                   [[-0.8564,  0.8907, -0.9702],
│                    [ 1.3878,  0.9042, -0.9959]],
│           
│                   [[-0.8564,  0.8907, -0.9702],
│                    [ 1.3878,  0.9042, -0.9959]],
│           
│                   [[-0.8564,  0.8907, -0.9702],
│                    [ 1.3878,  0.9042, -0.9959]],
│           
│                   [[-0.8564,  0.8907, -0.9702],
│                    [ 1.3878,  0.9042, -0.9959]],
│           
│                   [[-0.8564,  0.8907, -0.9702],
│                    [ 1.3878,  0.9042, -0.9959]],
│           
│                   [[-0.8564,  0.8907, -0.9702],
│                    [ 1.3878,  0.9042, -0.9959]],
│           
│                   [[-0.8564,  0.8907, -0.9702],
│                    [ 1.3878,  0.9042, -0.9959]]])
└── 'x' --> <FastTreeValue 0x7ff162396f10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.1 µs ± 97.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff169002700>
├── 'a' --> tensor([[-0.8564,  0.8907, -0.9702],
│                   [ 1.3878,  0.9042, -0.9959],
│                   [-0.8564,  0.8907, -0.9702],
│                   [ 1.3878,  0.9042, -0.9959],
│                   [-0.8564,  0.8907, -0.9702],
│                   [ 1.3878,  0.9042, -0.9959],
│                   [-0.8564,  0.8907, -0.9702],
│                   [ 1.3878,  0.9042, -0.9959],
│                   [-0.8564,  0.8907, -0.9702],
│                   [ 1.3878,  0.9042, -0.9959],
│                   [-0.8564,  0.8907, -0.9702],
│                   [ 1.3878,  0.9042, -0.9959],
│                   [-0.8564,  0.8907, -0.9702],
│                   [ 1.3878,  0.9042, -0.9959],
│                   [-0.8564,  0.8907, -0.9702],
│                   [ 1.3878,  0.9042, -0.9959]])
└── 'x' --> <FastTreeValue 0x7ff162394f40>
    └── 'c' --> tensor([[ 1.2597, -0.6361, -0.7958, -1.6428],
                        [-0.8940, -0.9450,  1.4342, -0.4739],
                 

In [23]:
%timeit t_cat(trees)

34.5 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.1 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2597, -0.6361, -0.7958, -1.6428],
                       [-0.8940, -0.9450,  1.4342, -0.4739],
                       [-1.9996, -1.7055, -0.7922, -0.5828]],
              
                      [[ 1.2597, -0.6361, -0.7958, -1.6428],
                       [-0.8940, -0.9450,  1.4342, -0.4739],
                       [-1.9996, -1.7055, -0.7922, -0.5828]],
              
                      [[ 1.2597, -0.6361, -0.7958, -1.6428],
                       [-0.8940, -0.9450,  1.4342, -0.4739],
                       [-1.9996, -1.7055, -0.7922, -0.5828]],
              
                      [[ 1.2597, -0.6361, -0.7958, -1.6428],
                       [-0.8940, -0.9450,  1.4342, -0.4739],
                       [-1.9996, -1.7055, -0.7922, -0.5828]],
              
                      [[ 1.2597, -0.6361, -0.7958, -1.6428],
                       [-0.8940, -0.9450,  1.4342, -0.4739],
                       [-1.9996, -1.7055, -0.7922, -0.5828]],

In [26]:
%timeit Batch.stack(batches)

91.6 µs ± 324 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2597, -0.6361, -0.7958, -1.6428],
                      [-0.8940, -0.9450,  1.4342, -0.4739],
                      [-1.9996, -1.7055, -0.7922, -0.5828],
                      [ 1.2597, -0.6361, -0.7958, -1.6428],
                      [-0.8940, -0.9450,  1.4342, -0.4739],
                      [-1.9996, -1.7055, -0.7922, -0.5828],
                      [ 1.2597, -0.6361, -0.7958, -1.6428],
                      [-0.8940, -0.9450,  1.4342, -0.4739],
                      [-1.9996, -1.7055, -0.7922, -0.5828],
                      [ 1.2597, -0.6361, -0.7958, -1.6428],
                      [-0.8940, -0.9450,  1.4342, -0.4739],
                      [-1.9996, -1.7055, -0.7922, -0.5828],
                      [ 1.2597, -0.6361, -0.7958, -1.6428],
                      [-0.8940, -0.9450,  1.4342, -0.4739],
                      [-1.9996, -1.7055, -0.7922, -0.5828],
                      [ 1.2597, -0.6361, -0.7958, -1.6428],
                   

In [28]:
%timeit Batch.cat(batches)

165 µs ± 537 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

391 µs ± 4.58 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
